# 4. Detect Nodules from Kaggle Dataset

## Summary
* load and process kaggle dataset
* Generate prediction masks with trained unet
* Reduce false positives with trained CNN

In [5]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
### EDIT HERE ###

unetweightspath="modelweights/unet-weights-improvement.hdf5"
truenoduleweightspath="modelweights/truenodule-cnn-weights-improvement.hdf5"
INPUT_FOLDER = 'stage1/' #path to kaggle stage1 dataset
datafolder="processeddata/"

####################
import numpy as np # linear algebra

import matplotlib.pyplot as plt
import dicom
import os
import scipy.ndimage
import time
from keras.callbacks import ModelCheckpoint
import h5py
from sklearn.cluster import KMeans
from skimage import measure, morphology
import cell_magic_wand as cmw
import numpy as np
import csv
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import confusion_matrix, classification_report
from scipy.ndimage.measurements import center_of_mass, label
from skimage.measure import regionprops

import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, SpatialDropout2D
from keras.layers import Input, merge, UpSampling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.models import Sequential
from keras import backend as K
from keras.optimizers import Adam
# Some constants 

patients = os.listdir(INPUT_FOLDER)
#patients=patients.sort()
K.set_image_dim_ordering('th') 

#Code sourced from https://www.kaggle.com/c/data-science-bowl-2017#tutorial
smooth = 1.0
width = 32

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def unet_model():
    inputs = Input((1, 512, 512))
    conv1 = Conv2D(width, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Conv2D(width, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(width*2, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = BatchNormalization(axis = 1)(conv2)
    conv2 = Conv2D(width*2, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(width*4, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = BatchNormalization(axis = 1)(conv3)
    conv3 = Conv2D(width*4, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(width*8, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = BatchNormalization(axis = 1)(conv4)
    conv4 = Conv2D(width*8, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(width*16, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = BatchNormalization(axis = 1)(conv5)
    conv5 = Conv2D(width*16, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = SpatialDropout2D(0.35)(up6)
    conv6 = Conv2D(width*8, 3, 3, activation='relu', border_mode='same')(conv6)
    conv6 = Conv2D(width*8, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = SpatialDropout2D(0.35)(up7)
    conv7 = Conv2D(width*4, 3, 3, activation='relu', border_mode='same')(conv7)
    conv7 = Conv2D(width*4, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = SpatialDropout2D(0.35)(up8)
    conv8 = Conv2D(width*2, 3, 3, activation='relu', border_mode='same')(conv8)
    conv8 = Conv2D(width*2, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = SpatialDropout2D(0.35)(up9)
    conv9 = Conv2D(width, 3, 3, activation='relu', border_mode='same')(conv9)
    conv9 = Conv2D(width, 3, 3, activation='relu', border_mode='same')(conv9)
    conv10 = Conv2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
    return model

unet_model=unet_model()
unet_model.load_weights(unetweightspath)

c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:64: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:71: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:74: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", padding="same")`

In [4]:
#classify as nodule or non-nodule
input_shape=(1,512,512)
num_classes=2
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])


model.load_weights(truenoduleweightspath)
#os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
#plot_model(model, to_file="CNNdiagram.png")

In [24]:
# Load the scans in given folder path
def load_scan(path):
    # code sourced from https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial
    slices = [dicom.read_file(path + '/' + s, force=True) for s in os.listdir(path) if s.endswith('.dcm')]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]), reverse=True)
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(slices):
    #code sourced from https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

def processimage(img):
    #function sourced from https://www.kaggle.com/c/data-science-bowl-2017#tutorial
    #Standardize the pixel values
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    #plt.hist(img.flatten(),bins=200)
    #plt.show()
    #print(thresh_img[366][280:450])
    middle = img[100:400,100:400] 
    mean = np.mean(middle)  
    max = np.max(img)
    min = np.min(img)
    #move the underflow bins
    img[img==max]=mean
    img[img==min]=mean
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image
    eroded = morphology.erosion(thresh_img,np.ones([4,4]))
    dilation = morphology.dilation(eroded,np.ones([10,10]))
    labels = measure.label(dilation)
    label_vals = np.unique(labels)
    #plt.imshow(labels)
    #plt.show()
    labels = measure.label(dilation)
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0]<475 and B[3]-B[1]<475 and B[0]>40 and B[2]<472:
            good_labels.append(prop.label)
    mask = np.ndarray([512,512],dtype=np.int8)
    mask[:] = 0
    #
    #  The mask here is the mask for the lungs--not the nodes
    #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask 
    #
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation
    return mask*img

def processimagenomask(img):
    #Standardize the pixel values
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    #plt.hist(img.flatten(),bins=200)
    #plt.show()
    #print(thresh_img[366][280:450])
    middle = img[100:400,100:400] 
    mean = np.mean(middle)  
    max = np.max(img)
    min = np.min(img)
    #move the underflow bins
    img[img==max]=mean
    img[img==min]=mean
    return img

def processimagefromfile(ppix):
    processpix=np.ndarray([ppix.shape[0],1,512,512])
    for i in range(ppix.shape[0]):
        processpix[i,0]=processimage(ppix[i])
    return processpix

#predict mask from images
def predictmask(images):
    num_test=images.shape[0]
    imgs_mask_test = np.ndarray([num_test,1,512,512],dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = unet_model.predict([images[i:i+1]], verbose=0)[0]
    return imgs_mask_test

#find number of slices where a nodule is detected
def getnoduleindex(imgs_mask_test):
    masksum=[np.sum(maskslice[0]) for maskslice in imgs_mask_test]
    return [i for i in range(len(masksum)) if masksum[i]>5]

def trueindicies(processed_pix, noduleindex):
    noduleimgs=[processed_pix[ind] for ind in noduleindex]
    noduleimgs=np.array(noduleimgs)
    predictions=model.predict(noduleimgs)
    predictions=predictions[:len(predictions),1]
    predictions[predictions>0.5]=True
    predictions[predictions<0.5]=False
    trueindicies=[ind for i,ind in enumerate(noduleindex) if predictions[i]==1]
    return trueindicies

def thresholdnodules(noduleindices,mask):
    nodulearea=[]
    for ind in trueindicies:
        nodulearea.append(np.sum(mask[ind]))
    return nodulearea

def largestnodulecoordinates(mask):
    #mask=nodulemasks[indx,0][0]
    mask[mask>0.5]=1
    mask[mask<0.5]=0
    labeled_array,nf=label(mask)
    areasinslice=[]
    if nf>1:
        for n in range(nf):
            lab=np.array(labeled_array)
            lab[lab!=(n+1)]=0
            lab[lab==(n+1)]=1
            areasinslice.append(np.sum(lab))
        nlargest=areasinslice.index(max(areasinslice))
        labeled_array[labeled_array!=(nlargest+1)]=0
        com=center_of_mass(labeled_array)
    else:
        com=center_of_mass(mask)
    return [int(com[0]),int(com[1])]

def largestnodulearea(mask,table,i):
    #mask=nodulemasks[indx,0][0]
    mask[mask>0.5]=1
    mask[mask<0.5]=0
    labeled_array,nf=label(mask)
    areasinslice=[]
    if nf>1:
        for n in range(nf):
            lab=np.array(labeled_array)
            lab[lab!=(n+1)]=0
            lab[lab==(n+1)]=1
            areasinslice.append(np.sum(lab))
        #nlargest=areasinslice.index(max(areasinslice))
        #labeled_array[labeled_array!=(nlargest+1)]=0
        return max(areasinslice)
    else:
        return table["Area"][i]

def crop_nodule(coord,image):
    dim=32
    return image[coord[0]-dim:coord[0]+dim,coord[1]-dim:coord[1]+dim]
#output: 64x64 images of the nodules with malignancy labels from the patient

def largestnoduleproperties(mask):
    mask[mask>0.5]=1
    mask[mask<0.5]=0
    mask=mask.astype(np.int8)
    labeled_array,nf=label(mask)
    areasinslice=[]
    if nf>1:
        for n in range(nf):
            lab=np.array(labeled_array)
            lab[lab!=(n+1)]=0
            lab[lab==(n+1)]=1
            areasinslice.append(np.sum(lab))
        nlargest=areasinslice.index(max(areasinslice))
        labeled_array[labeled_array!=(nlargest+1)]=0
        noduleprops=regionprops(labeled_array)
    else:
        noduleprops=regionprops(mask)
    area=noduleprops[0].area
    eccentricity=noduleprops[0].eccentricity
    diam=noduleprops[0].equivalent_diameter
    diammajor=noduleprops[0].major_axis_length
    spiculation=noduleprops[0].solidity
    return area, eccentricity, diam, diammajor, spiculation

def generatefeaturetable(nodulemasks):
    meannoduleHU=[]
    nodulecount=[]
    largestarealist=[]
    eccentricitylist=[]
    diamlist=[]
    diammajorlist=[]
    spiculationlist=[]

    for i in range(nodulemasks.shape[0]):
        mask=nodulemasks[i,0]
        mask[mask>0.5]=1
        mask[mask<0.5]=0
        meannoduleHU.append(np.sum(noduleimages[i,0]*mask)/np.sum(mask))
        labeled_array,features=label(mask)
        nodulecount.append(features)
        area, eccentricity, diam, diammajor, spiculation = largestnoduleproperties(nodulemasks[i,0])
        largestarealist.append(area)
        eccentricitylist.append(eccentricity)
        diamlist.append(diam)
        diammajorlist.append(diammajor)
        spiculationlist.append(spiculation)
    table=pd.DataFrame({"Patient":sample,"NoduleIndex":noduleindicies,"Area":area,"MeanHU":meannoduleHU, "LargestNoduleArea":largestarealist,
                    "Eccentricity":eccentricitylist, "Diameter":diamlist, "DiameterMajor":diammajorlist, "Spiculation":spiculationlist})
    return table

In [6]:
start_time=time.time()

elapsed_time=0
totaltime=94000
thresh=-500 #lower HU threshold for nodule segmentation
noduleimages=np.ndarray([5000,1,512,512],dtype=np.float32)
nodulemasks=np.ndarray([5000,1,512,512],dtype=np.float32)
sample=[]
area=[]
noduleindicies=[]
index=0
start=1
end=400
for i in range(len(patients)):
    print("Processing patient#",i,"ETA:",(totaltime-elapsed_time)/3600,"hrs")
    if (i-1)/400-np.floor((i-1)/400)==0:
        noduleimages=noduleimages[:index]
        nodulemasks=nodulemasks[:index]
        table=generatefeaturetable(nodulemasks)
        print("Saving data for patients"+str(start)+"-"+str(end))
        np.save(datafolder+"DSBNoduleImages"+str(start)+"-"+str(end)+".npy",noduleimages)
        np.save(datafolder+"DSBNoduleMasks"+str(start)+"-"+str(end)+".npy",nodulemasks)
        table.to_csv(datafolder+"DSBNoduleFeatures"+str(start)+"-"+str(end)+".csv")
        del noduleimages, nodulemasks
        noduleimages=np.ndarray([5000,1,512,512],dtype=np.float32)
        nodulemasks=np.ndarray([5000,1,512,512],dtype=np.float32)
        sample=[]
        area=[]
        noduleindicies=[]
        index=0  
    patient_scan=load_scan(INPUT_FOLDER+patients[i])
    patient_pix=get_pixels_hu(patient_scan)
    processed_pix = processimagefromfile(patient_pix)
    mask = predictmask(processed_pix)
    noduleindex = getnoduleindex(mask)
    trueinds=trueindicies(processed_pix,noduleindex)

    for ind in trueinds:
        noduleimages[index,0]=patient_pix[ind]
        nodulemasks[index,0]=mask[ind]
        sample.append(patients[i])
        area.append(np.sum(mask[ind]))
        noduleindicies.append(ind)
        index+=1

    elapsed_time=time.time()-start_time
    totaltime=elapsed_time/(i-start+1)*(end-start)



print(elapsed_time)

Processing patient# 1200 ETA: 13.88888888888889 hrs
Processing patient# 1201 ETA: 12.623715912898382 hrs
Processing patient# 1202 ETA: 10.018525106410186 hrs
Processing patient# 1203 ETA: 18.164322644339666 hrs
Processing patient# 1204 ETA: 15.95403834571441 hrs
Processing patient# 1205 ETA: 14.166520277659098 hrs
Processing patient# 1206 ETA: 13.91482162181978 hrs
Processing patient# 1208 ETA: 13.044705638507056 hrs
Processing patient# 1209 ETA: 11.268210861565155 hrs
Processing patient# 1210 ETA: 11.153029166426924 hrs
Processing patient# 1211 ETA: 10.874260760220615 hrs
Processing patient# 1212 ETA: 10.507523334208463 hrs
Processing patient# 1213 ETA: 10.314581797112767 hrs
Processing patient# 1214 ETA: 10.08706813660406 hrs
Processing patient# 1216 ETA: 9.996492877094832 hrs
Processing patient# 1217 ETA: 9.102958305653404 hrs
Processing patient# 1219 ETA: 8.925979718978024 hrs
Processing patient# 1220 ETA: 8.507819953064123 hrs
Processing patient# 1221 ETA: 8.645915125431838 hrs
Pr

Processing patient# 1385 ETA: 4.570799427429835 hrs
Processing patient# 1386 ETA: 4.54748295711004 hrs
Processing patient# 1387 ETA: 4.527965035849359 hrs
Processing patient# 1388 ETA: 4.499852827242081 hrs
Processing patient# 1389 ETA: 4.477454396751192 hrs
Processing patient# 1390 ETA: 4.456050586798038 hrs
Processing patient# 1391 ETA: 4.434381366932787 hrs
Processing patient# 1392 ETA: 4.407015767265111 hrs
Processing patient# 1394 ETA: 4.37906760837885 hrs
Processing patient# 1395 ETA: 4.30689841369618 hrs
Processing patient# 1396 ETA: 4.285369354544922 hrs
Processing patient# 1397 ETA: 4.268326153216628 hrs
Processing patient# 1398 ETA: 4.263744453107467 hrs
Processing patient# 1399 ETA: 4.243092987329717 hrs
Processing patient# 1400 ETA: 4.222136327758431 hrs
Processing patient# 1401 ETA: 4.198838181790889 hrs
Processing patient# 1402 ETA: 4.1793994686896 hrs
Processing patient# 1403 ETA: 4.165873683685152 hrs
Processing patient# 1404 ETA: 4.145204392498549 hrs
Processing patien

Processing patient# 1561 ETA: 0.7496390868227617 hrs
Processing patient# 1562 ETA: 0.7277910622262829 hrs
Processing patient# 1563 ETA: 0.70534063768343 hrs
Processing patient# 1564 ETA: 0.6836017272867927 hrs
Processing patient# 1566 ETA: 0.6618160449642033 hrs
Processing patient# 1568 ETA: 0.6160384095808674 hrs
Processing patient# 1570 ETA: 0.5702290712717382 hrs
Processing patient# 1571 ETA: 0.5249562931103638 hrs
Processing patient# 1572 ETA: 0.5033148116076286 hrs
Processing patient# 1573 ETA: 0.48112341017214527 hrs
Processing patient# 1574 ETA: 0.4594317513380257 hrs
Processing patient# 1575 ETA: 0.43778636997364123 hrs
Processing patient# 1576 ETA: 0.415779529424036 hrs
Processing patient# 1577 ETA: 0.3949588738670405 hrs
Processing patient# 1578 ETA: 0.37282882327230255 hrs
Processing patient# 1580 ETA: 0.3515907668564129 hrs
Processing patient# 1581 ETA: 0.30684781963013846 hrs
Processing patient# 1582 ETA: 0.28561606805205886 hrs
Processing patient# 1583 ETA: 0.263678909045